# IF3170 Artificial Intelligence | Tugas Besar 2

Nama Kelompok: EgiluyIzinnn🙏


Group Number: 34

Group Members:
- Nicholas Andhika Lucas (13523014)
- Shanice Feodora Tjahjono (13523097)
- Jonathan Kenan Budianto (13523139)
- Mahesa Fadhillah Andre (13523140)
- Muhammad Farrel Wibowo (13523153)

## Import Libraries

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer

## Import Dataset

In [71]:
# Example of reading a csv file from a gdrive link

# Take the file id from the gdrive file url
# https://drive.google.com/file/d/1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU/view?usp=sharing => The file id is 1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU
# and then put it in this format:
# https://drive.google.com/uc?id={file_id}
# Don't forget to change the access to public

df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

print(f"Train shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")

Train shape: (3096, 38)
Test shape: (1328, 37)


# Exploratory Data Analysis (Optional)

Exploratory Data Analysis (EDA) is a crucial step in the data analysis process that involves examining and visualizing data sets to uncover patterns, trends, anomalies, and insights. It is the first step before applying more advanced statistical and machine learning techniques. EDA helps you to gain a deep understanding of the data you are working with, allowing you to make informed decisions and formulate hypotheses for further analysis.

In [72]:
print("EXPLORATORY DATA ANALYSIS")

# Dataset Information
print("\nInformasi Dataset:") 
df_train.info()

# Ringkasan Statistik
print("\nRingkasan Statistik:")
print(df_train.describe())

# Cek missing values
print("\nMissing Values:")
print(df_train.isnull().sum())
print(f"Total missing values: {df_train.isnull().sum().sum()}")

# Cek duplikasi
print(f"\nDuplicate rows: {df_train.duplicated().sum()}")

# Target Distribution
print("\nTarget Distribution:")
print(df_train['Target'].value_counts())
print("\nLabel Distribution (%):")
print(df_train['Target'].value_counts(normalize=True) * 100)

# Visualisasi Data
# TODO


EXPLORATORY DATA ANALYSIS

Informasi Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096 entries, 0 to 3095
Data columns (total 38 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Student_ID                                      3096 non-null   int64  
 1   Marital status                                  3096 non-null   int64  
 2   Application mode                                3096 non-null   int64  
 3   Application order                               3096 non-null   int64  
 4   Course                                          3096 non-null   int64  
 5   Daytime/evening attendance	                     3096 non-null   int64  
 6   Previous qualification                          3096 non-null   int64  
 7   Previous qualification (grade)                  3096 non-null   float64
 8   Nacionality                                     3096 non-null   int64  


# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [73]:
# Split training set and validation set here, store into variables train_set and val_set.
# Remember to also keep the original training set before splitting. This will come important later.

# Training and validation split on 80/20 ratio
train_set, val_set = train_test_split(
    df_train, 
    test_size=0.2, 
    random_state=42, 
    stratify=df_train['Target']
)

print("\nTarget Distribution:")
print("\nFull Training Data:")
print(df_train['Target'].value_counts())
print("\nTraining Set (80%):")
print(train_set['Target'].value_counts())
print("\nValidation Set (20%):")
print(val_set['Target'].value_counts())


Target Distribution:

Full Training Data:
Target
Graduate    1546
Dropout      994
Enrolled     556
Name: count, dtype: int64

Training Set (80%):
Target
Graduate    1236
Dropout      795
Enrolled     445
Name: count, dtype: int64

Validation Set (20%):
Target
Graduate    310
Dropout     199
Enrolled    111
Name: count, dtype: int64


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [74]:
# There is no missing data in EDA, skip this process

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [75]:
numerical_cols = train_set.select_dtypes(include=['number']).columns

# Exclude ID, target and categorical/binary columns
exclude_cols = [
    'Student_ID', 'Target',
    'Marital status', 'Application mode', 'Application order', 'Course',
    'Daytime/evening attendance\t', 'Previous qualification', 'Nacionality',
    "Mother's qualification", "Father's qualification",
    "Mother's occupation", "Father's occupation",
    'Displaced', 'Educational special needs', 'Debtor',
    'Tuition fees up to date', 'Gender', 'Scholarship holder', 'International'
]

cols_to_process = [c for c in numerical_cols if c not in exclude_cols]

# Check for outliers using IQR
print("\nOutlier Detection with IQR Method:")

for col in cols_to_process:
    Q1 = train_set[col].quantile(0.25)
    Q3 = train_set[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_count = ((train_set[col] < lower_bound) | (train_set[col] > upper_bound)).sum()
    outliers_percent = outliers_count / len(train_set) * 100

    if outliers_count > 0:
        print(f"Column {col}: {outliers_count} outliers ({outliers_percent:.2f}%)")
    
    if 'grade' in col.lower():
        # Keep grade outliers as they represent valid high/low performance
        continue

    elif outliers_percent > 10:
        # Apply Log Transformation for high frequency outliers (>10%)
        train_set[col] = train_set[col].apply(lambda x: np.log1p(x) if x > 0 else x)
        val_set[col] = val_set[col].apply(lambda x: np.log1p(x) if x > 0 else x)
    elif outliers_percent > 0:
        # Apply Capping for low frequency outliers (<10%)
        train_set[col] = np.clip(train_set[col], lower_bound, upper_bound)
        val_set[col] = np.clip(val_set[col], lower_bound, upper_bound)

print("\nOutlier Handling Complete")


Outlier Detection with IQR Method:
Column Previous qualification (grade): 95 outliers (3.84%)
Column Admission grade: 46 outliers (1.86%)
Column Age at enrollment: 251 outliers (10.14%)
Column Curricular units 1st sem (credited): 329 outliers (13.29%)
Column Curricular units 1st sem (enrolled): 246 outliers (9.94%)
Column Curricular units 1st sem (evaluations): 87 outliers (3.51%)
Column Curricular units 1st sem (approved): 99 outliers (4.00%)
Column Curricular units 1st sem (grade): 406 outliers (16.40%)
Column Curricular units 1st sem (without evaluations): 174 outliers (7.03%)
Column Curricular units 2nd sem (credited): 301 outliers (12.16%)
Column Curricular units 2nd sem (enrolled): 215 outliers (8.68%)
Column Curricular units 2nd sem (evaluations): 59 outliers (2.38%)
Column Curricular units 2nd sem (approved): 27 outliers (1.09%)
Column Curricular units 2nd sem (grade): 494 outliers (19.95%)
Column Curricular units 2nd sem (without evaluations): 153 outliers (6.18%)

Outlier Ha

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [76]:
# There is no duplicate data in EDA, skip this process

### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [77]:
print("Applying Feature Engineering...")

train_set = train_set.copy()
val_set = val_set.copy()

# *** 1. CREATE INTERACTION FEATURES: Academic Performance Metrics ***
print("\n1. Creating Interaction Features...")

# Total ennrolled courses from 2 semesters
train_set['Total_enrolled'] = train_set['Curricular units 1st sem (enrolled)'] + train_set['Curricular units 2nd sem (enrolled)']
val_set['Total_enrolled'] = val_set['Curricular units 1st sem (enrolled)'] + val_set['Curricular units 2nd sem (enrolled)']

# Total approved units from 2 semesters
train_set['Total_approved'] = train_set['Curricular units 1st sem (approved)'] + train_set['Curricular units 2nd sem (approved)']
val_set['Total_approved'] = val_set['Curricular units 1st sem (approved)'] + val_set['Curricular units 2nd sem (approved)']

# Total credited units
train_set['Total_credited'] = train_set['Curricular units 1st sem (credited)'] + train_set['Curricular units 2nd sem (credited)']
val_set['Total_credited'] = val_set['Curricular units 1st sem (credited)'] + val_set['Curricular units 2nd sem (credited)']

# Average grade from 2 semesters
train_set['Average_grade'] = (train_set['Curricular units 1st sem (grade)'] + train_set['Curricular units 2nd sem (grade)']) / 2
val_set['Average_grade'] = (val_set['Curricular units 1st sem (grade)'] + val_set['Curricular units 2nd sem (grade)']) / 2

# Approved rate from 1st semester
train_set['Approval_rate_sem1'] = train_set.apply(
    lambda row: row['Curricular units 1st sem (approved)'] / row['Curricular units 1st sem (enrolled)'] 
    if row['Curricular units 1st sem (enrolled)'] > 0 else 0, axis=1
)
val_set['Approval_rate_sem1'] = val_set.apply(
    lambda row: row['Curricular units 1st sem (approved)'] / row['Curricular units 1st sem (enrolled)'] 
    if row['Curricular units 1st sem (enrolled)'] > 0 else 0, axis=1
)

# Approved rate from 2nd semester
train_set['Approval_rate_sem2'] = train_set.apply(
    lambda row: row['Curricular units 2nd sem (approved)'] / row['Curricular units 2nd sem (enrolled)'] 
    if row['Curricular units 2nd sem (enrolled)'] > 0 else 0, axis=1
)
val_set['Approval_rate_sem2'] = val_set.apply(
    lambda row: row['Curricular units 2nd sem (approved)'] / row['Curricular units 2nd sem (enrolled)'] 
    if row['Curricular units 2nd sem (enrolled)'] > 0 else 0, axis=1
)

# Grade improvement from 1st to 2nd semester
train_set['Grade_improvement'] = train_set['Curricular units 2nd sem (grade)'] - train_set['Curricular units 1st sem (grade)']
val_set['Grade_improvement'] = val_set['Curricular units 2nd sem (grade)'] - val_set['Curricular units 1st sem (grade)']

print("Created interaction features")

# *** 2. AGE BINNING (Domain-Based: <20, 20-25, >25) ***
print("\n2. Applying Age Binning...")
# Young (<20)
# Traditional (20-25)
# Mature (>25)
def bin_age(age):
    if age < 20:
        return 0  # Young
    elif age <= 25:
        return 1  # Traditional
    else:
        return 2  # Mature

train_set['Age_group'] = train_set['Age at enrollment'].apply(bin_age)
val_set['Age_group'] = val_set['Age at enrollment'].apply(bin_age)

print("Created Age_group feature (0=Young, 1=Traditional, 2=Mature)")

# *** 3. FEATURE SELECTION (Remove low correlation <0.05) ***
print("\n3. Performing Feature Selection...")

# Calculate correlations
le = LabelEncoder()
target_encoded = le.fit_transform(train_set['Target'])
numeric_features = train_set.select_dtypes(include=['number']).columns
numeric_features = [c for c in numeric_features if c not in ['Student_ID', 'Target']]

correlations = {}
for col in numeric_features:
    corr = abs(np.corrcoef(train_set[col], target_encoded)[0, 1])
    correlations[col] = corr

# Identify features to remove (correlation < 0.05)
# Exception: Keep Age and Admission grade because they may have non-linear relationships, e.g. lower grade might indicate a certain target
# Reasoning: correlations below 5% almost have no linear relationship with the target and is negligible
features_to_remove = []
exempt_features = ['Age at enrollment', 'Admission grade']

for feature, corr in correlations.items():
    if corr < 0.05 and feature not in exempt_features:
        features_to_remove.append(feature)

# Remove low correlation features
if features_to_remove:
    print(f"Removing {len(features_to_remove)} low-correlation features:")
    for feature in features_to_remove:
        print(f"      - {feature} (corr={correlations[feature]:.4f})")
    train_set = train_set.drop(columns=features_to_remove)
    val_set = val_set.drop(columns=features_to_remove)
else:
    print(f"No features to remove")

# *** NOTE: May be tested after first training to see if this improves the model ***
# *** 4. POLYNOMIAL FEATURES (Capture Non-linear Relationships) ***
# print("\n4. Adding Polynomial Features...")

# # Age^2 - captures U-shaped relationship (young & old students struggle more)
# train_set['Age_squared'] = train_set['Age at enrollment'] ** 2
# val_set['Age_squared'] = val_set['Age at enrollment'] ** 2

# # Admission_grade^2 - captures threshold effects (diminishing returns at high grades)
# train_set['Admission_grade_squared'] = train_set['Admission grade'] ** 2
# val_set['Admission_grade_squared'] = val_set['Admission grade'] ** 2

# # Top 3 correlated features squared (if not already polynomial)
# top_features = sorted(correlations.items(), key=lambda x: x[1], reverse=True)[:3]
# poly_count = 2  # Already added Age and Admission

# for feature, corr in top_features:
#     if feature not in ['Age at enrollment', 'Admission grade'] and feature in train_set.columns:
#         train_set[f'{feature}_squared'] = train_set[feature] ** 2
#         val_set[f'{feature}_squared'] = val_set[feature] ** 2
#         print(f"   Added {feature}_squared (corr={corr:.4f})")
#         poly_count += 1
#         if poly_count >= 5:  # Limit to 5 polynomial features to avoid curse of dimensionality
#             break

# print(f"   Created {poly_count} polynomial features")

# SUMMARY
print(f"\nFinal feature count: {len(train_set.columns)}")
print(f"   - Original: {len(df_train.columns)}")
print(f"   - Added: 8 interactions + 1 age bin")
# print(f"   - Added: 8 interactions + 1 age bin + {poly_count} polynomial = {9 + poly_count}")
print(f"   - Removed: {len(features_to_remove)} low-correlation features")
print("="*70)

Applying Feature Engineering...

1. Creating Interaction Features...
Created interaction features

2. Applying Age Binning...
Created Age_group feature (0=Young, 1=Traditional, 2=Mature)

3. Performing Feature Selection...
Removing 15 low-correlation features:
      - Course (corr=0.0316)
      - Previous qualification (corr=0.0385)
      - Nacionality (corr=0.0082)
      - Father's qualification (corr=0.0205)
      - Mother's occupation (corr=0.0106)
      - Father's occupation (corr=0.0020)
      - Educational special needs (corr=0.0027)
      - International (corr=0.0154)
      - Curricular units 1st sem (credited) (corr=0.0326)
      - Curricular units 1st sem (evaluations) (corr=0.0243)
      - Curricular units 2nd sem (credited) (corr=0.0429)
      - Unemployment rate (corr=0.0028)
      - Inflation rate (corr=0.0268)
      - GDP (corr=0.0407)
      - Total_credited (corr=0.0379)

Final feature count: 31
   - Original: 38
   - Added: 8 interactions + 1 age bin
   - Removed: 15 lo

c:\Users\reala\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\reala\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [78]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [79]:
# Write your code here

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [80]:
# Write your code here

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [81]:
# Write your code here

### IV. Data Normalization

Data normalization is used to achieve a standard distribution. Without normalization, models or processes that rely on the assumption of normality may not work correctly. Normalization helps reduce the magnitude effect and ensures numerical stability during optimization.

In [82]:
# Write your code here

### V. Dimensionality Reduction

Dimensionality reduction is a technique used in data preprocessing to reduce the number of input features (dimensions) in a dataset while retaining as much important information as possible. It is essential when dealing with high-dimensional data, where too many features can cause problems like increased computational costs, overfitting, and difficulty in visualization. Reducing dimensions simplifies the data, making it easier to analyze and improving the performance of machine learning models.

One of the main approaches to dimensionality reduction is feature extraction. Feature extraction creates new, smaller sets of features that capture the essence of the original data. Common techniques include:

1. **Principal Component Analysis (PCA)**: Converts correlated features into a smaller number of uncorrelated "principal components."
2. **t-SNE (t-Distributed Stochastic Neighbor Embedding)**: A visualization-focused method to project high-dimensional data into 2D or 3D spaces.
3. **Autoencoders**: Neural networks that learn compressed representations of the data.

In [83]:
# Write your code here

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [84]:
# from sklearn.pipeline import Pipeline

# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("imputer", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [85]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [86]:
# Write your code here

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `attack_cat`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. DTL

In [87]:
# Type your code here

## B. Logistic Regression

In [88]:
# Type your code here

## C. SVM

In [89]:
# Type your code here

## D. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [90]:
# Type your code here

## E. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `attack_cat` column.

In [91]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`